In [ ]:
from google.colab import drive

drive.mount("gdrive")

%cd gdrive/MyDrive/aitalenthack/

Mounted at gdrive
/content/gdrive/MyDrive/aitalenthack


## Load dataset

In [ ]:
def read_dataset(file_name):
    try:
        import pickle

    except:
        !pip install pickle

    with open(file_name, "rb") as handle:
        loaded_file = pickle.load(handle)

    return loaded_file


dataset = read_dataset("unsplash_images/interier_dataset.pickle")
dataset_second_part = read_dataset("unsplash_images/interier_second_part.pickle")

dataset.extend(dataset_second_part)

print(f"Data shape: {len(dataset)}")


Data shape: 2364


In [ ]:
import json

with open("unsplash_images/metadata.jsonl", "w") as file:
    for line in dataset:
        # rename key cause imagefolder works only with certain naming
        line["file_name"] = line.pop("image")
        file.write(json.dumps(line) + '\n')


In [ ]:
try:
    from datasets import load_dataset

except:
    !pip install datasets -U -q
    from datasets import load_dataset

dataset = load_dataset("imagefolder", data_dir="unsplash_images/", split="train")


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.1/542.1 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.31.0, but you have requests 2.32.3 which is incompatible.


Resolving data files:   0%|          | 0/2367 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
dataset.save_to_disk("/content/gdrive/MyDrive/aitalenthack/unsplash_dataset.hf")


Saving the dataset (0/9 shards):   0%|          | 0/2364 [00:00<?, ? examples/s]

## Fine-tune

In [ ]:
%cd /content/
!git clone https://github.com/huggingface/diffusers.git
!pip install -e diffusers/ -q
!ls -la


!pip install -r diffusers/examples/text_to_image/requirements_sdxl.txt -q
!pip install accelerate -q
!pip install wandb -q
!pip install -U peft -q
!pip install transformers -q
!pip install xformers -q

!accelerate config default


/content
Cloning into 'diffusers'...
remote: Enumerating objects: 63744, done.
remote: Counting objects: 100% (259/259), done.
remote: Compressing objects: 100% (147/147), done.
remote: Total 63744 (delta 163), reused 180 (delta 101), pack-reused 63485
Receiving objects: 100% (63744/63744), 47.39 MiB | 31.70 MiB/s, done.
Resolving deltas: 100% (46663/46663), done.
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for diffusers (pyproject.toml) ... done
total 24
drwxr-xr-x  1 root root 4096 Jun 10 07:20 .
drwxr-xr-x  1 root root 4096 Jun 10 07:18 ..
drwxr-xr-x  4 root root 4096 Jun  6 14:15 .config
drwxr-xr-x 12 root root 4096 Jun 10 07:20 diffusers
drwx------  5 root root 4096 Jun 10 07:20 gdrive
drwxr-xr-x  1 root root 4096 Jun  6 14:21 sample_data
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 4

In [ ]:
from huggingface_hub import snapshot_download

snapshot_download(repo_id="stabilityai/stable-diffusion-xl-base-1.0", local_dir="/content")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 57 files:   0%|          | 0/57 [00:00<?, ?it/s]

scheduler/scheduler_config.json:   0%|          | 0.00/479 [00:00<?, ?B/s]

model_index.json:   0%|          | 0.00/609 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/8.67k [00:00<?, ?B/s]

...

vae_encoder/config.json:   0%|          | 0.00/607 [00:00<?, ?B/s]

model.onnx:   0%|          | 0.00/137M [00:00<?, ?B/s]

openvino_model.bin:   0%|          | 0.00/137M [00:00<?, ?B/s]

openvino_model.xml:   0%|          | 0.00/850k [00:00<?, ?B/s]

'/content'

In [ ]:
!HF_DATASETS_OFFLINE=1 TRANSFORMERS_OFFLINE=1 \
accelerate launch --mixed_precision="fp16" diffusers/examples/text_to_image/train_text_to_image_lora.py \
    --pretrained_model_name_or_path="/content" \
    --dataset_name="/content/gdrive/MyDrive/aitalenthack/unsplash_images" \
    --image_column="image" \
    --caption_column="text" \
    --validation_prompt="living room with a couch, chair, and a table" \
    --num_validation_images=4 \
    --resolution=1024 \
    --random_flip \
    --center_crop \
    --enable_xformers_memory_efficient_attention \
    --train_batch_size=5 \
    --max_train_steps=100 \
    --checkpointing_steps=80 \
    --checkpoints_total_limit=2 \
    --learning_rate=1e-04 \
    --lr_scheduler="linear" \
    --lr_warmup_steps=10 \
    --seed=42 \
    --output_dir="/content/gdrive/MyDrive/aitalenthack/sdxl_lora_weights" \
    --report_to="wandb"


/content/diffusers/src/diffusers/models/transformers/transformer_2d.py:34: FutureWarning: `Transformer2DModelOutput` is deprecated and will be removed in version 1.0.0. Importing `Transformer2DModelOutput` from `diffusers.models.transformer_2d` is deprecated and this will be removed in a future version. Please use `from diffusers.models.modeling_outputs import Transformer2DModelOutput`, instead.
  deprecate("Transformer2DModelOutput", "1.0.0", deprecation_message)
2024-06-08 12:48:56.632018: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-08 12:48:56.686009: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-08 12:48:56.686053